In [1]:
#instalando a biblioteca Llama index
!pip install -q llama-index


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Program Files\Python313\python.exe -m pip install --upgrade pip


In [2]:
# biblioteca para permitir a leitura de arquivos
from llama_index.core import SimpleDirectoryReader

In [3]:
documentos = SimpleDirectoryReader(input_dir='../Aula 7/arquivo')

In [4]:
documentos.input_files

[WindowsPath('c:/Users/47341687843/Documents/ChatBot/ChatBot/Aula 7/../Aula 7/arquivo/doc_info_agricultura.pdf')]

In [5]:
docs = documentos.load_data()

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)


In [6]:
docs[0].__dict__

{'id_': '659cc045-ee3b-43d2-b703-c9283e5096f4',
 'embedding': None,
 'metadata': {'page_label': '1',
  'file_name': 'doc_info_agricultura.pdf',
  'file_path': 'c:\\Users\\47341687843\\Documents\\ChatBot\\ChatBot\\Aula 7\\..\\Aula 7\\arquivo\\doc_info_agricultura.pdf',
  'file_type': 'application/pdf',
  'file_size': 30901,
  'creation_date': '2025-03-19',
  'last_modified_date': '2025-03-19'},
 'excluded_embed_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'excluded_llm_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'relationships': {},
 'metadata_template': '{key}: {value}',
 'metadata_separator': '\n',
 'text_resource': MediaResource(embeddings=None, data=None, text='Guia de Preparo do Solo para Plantio Informações Gerais P: Qual a importância do preparo adequado do solo? R: O preparo adequado do solo garante que as planta

In [7]:
# importando a biblioteca
from llama_index.core.node_parser import SentenceSplitter
node_parser= SentenceSplitter(chunk_size=1200)

In [8]:
nodes = node_parser.get_nodes_from_documents(docs,show_progress=True)

C:\Users\47341687843\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Parsing nodes: 100%|██████████| 3/3 [00:00<00:00, 1795.51it/s]


In [9]:
#instalaçao do hugging face
!pip install -q llama-index-embeddings-huggingface


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Program Files\Python313\python.exe -m pip install --upgrade pip


In [10]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [11]:
# Classe personalizada para integrar ao Chroma DB

class ChromaEmbeddingWrapper:
  def __init__(self,model_name:str): # inicializa com o modelo embedding
    self.model = HuggingFaceEmbedding(model_name = model_name)


  def __call__(self,input): # Converte a entrada para o formato compativel com o HuggingFace
    return self.model.embed(input)




In [12]:
embed_model_chroma= ChromaEmbeddingWrapper(model_name='intfloat/multilingual-e5-large')

In [18]:
#instalaçao do chroma db
#!pip install -q llama-index-vector-stores-chroma
# !pip install --upgrade pip
!pip install --upgrade chromadb


Defaulting to user installation because normal site-packages is not writeable

  error: subprocess-exited-with-error
  
  × Building wheel for chroma-hnswlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [5 lines of output]
      running bdist_wheel
      running build
      running build_ext
      building 'hnswlib' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for chroma-hnswlib

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Program Files\Python313\python.exe -m pip install --upgrade pip
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (chroma-hnswlib)



  Using cached chromadb-0.6.3-py3-none-any.whl.metadata (6.8 kB)
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached chroma_hnswlib-0.7.6.tar.gz (32 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached fastapi-0.115.11-py3-none-any.whl.metadata (27 kB)
  Using cached uvicorn-0.34.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached posthog-3.21.0-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached onnxruntime-1.21.0-cp313-cp313-win_amd64.whl.metadata (4.9 kB)
  Using cached opentelemetry_api-1.31.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.31.0-py3-none-any.whl.metadata (2.5 kB)
  Using cached opentelemetry_instrum

In [19]:
#importando chroma db
import chromadb
db = chromadb.PersistentClient(path='./chroma_db')
chroma_client = db
collection_name = 'documentos_serenatto'

try:
 chroma_collection = chroma_client.get_or_create_collection(
        name=collection_name,
        embedding_function=embed_model_chroma)

except Exception as e :
  print(f'Erro ao criar coleçao {e}')


ModuleNotFoundError: No module named 'chromadb'

In [ ]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

In [ ]:
#criando a coleçao do banco de dados
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
embed_model = HuggingFaceEmbedding(model_name='intfloat/multilingual-e5-large')

In [ ]:
from llama_index.core import VectorStoreIndex

In [ ]:
index = VectorStoreIndex(nodes, storage_context=storage_context,embed_model = embed_model)

In [ ]:
from llama_index.core import load_index_from_storage
index = load_index_from_storage(storage_context,embed_model = embed_model)

In [ ]:
from google.colab import userdata
Groq_api =userdata.get('api_groq_chat')

In [ ]:
!pip install -q llama-index-llms-groq

In [ ]:
from llama_index.llms.groq import Groq

In [ ]:
llms = Groq(model = 'llama3-70b-8192',api_key=Groq_api )

In [ ]:
query_engine = index.as_query_engine(llm=llms,similarity_top_k=2)

In [ ]:
query_engine.query('Quais graos estao disponiveis').response

'Trabalhamos com as seguintes variedades de café em grãos: Bourbon vermelho, Bourbon amarelo, Blend especial (Mistura de Bourbon amarelo e vermelho), Catuaí amarelo, Geisha, Yirgacheffe.'

In [ ]:
query_embedding = embed_model.get_text_embedding('Quais graos estao disponiveis ?')

In [ ]:
chroma_collection.query(query_embedding,n_results=2,include=['distances','embeddings'])

{'ids': [['3591a1e4-813c-45a4-b0eb-13ca928fb822',
   '757a9a46-5ba3-4356-97f9-b192fcce7a37']],
 'embeddings': [array([[ 0.01460024, -0.01119941, -0.00576889, ...,  0.00555894,
          -0.0503395 ,  0.00342874],
         [ 0.01460024, -0.01119941, -0.00576889, ...,  0.00555894,
          -0.0503395 ,  0.00342874]])],
 'documents': None,
 'uris': None,
 'data': None,
 'metadatas': None,
 'distances': [[0.3871879346745659, 0.3871879346745659]],
 'included': [<IncludeEnum.embeddings: 'embeddings'>,
  <IncludeEnum.distances: 'distances'>]}

In [ ]:
chat_engine = index.as_chat_engine(mode='context',llm = llms)

In [ ]:
pergunta = chat_engine.chat('Quais graos estao disponiveis ?').response
print(pergunta)

Os grãos disponíveis são Bourbon vermelho, Bourbon amarelo, Blend especial (uma mistura de Bourbon amarelo e vermelho), Catuaí amarelo, Geisha e Yirgacheffe.


'Os grãos disponíveis são: Bourbon vermelho, Bourbon amarelo, Blend especial (Mistura de Bourbon amarelo e vermelho), Catuaí amarelo, Geisha e Yirgacheffe.'

In [ ]:
pergunta = chat_engine.chat('Quais sao os preços dos graos ?').response
print(pergunta)

Os preços dos grãos são: Bourbon vermelho: R$ 41,00, Bourbon amarelo: R$ 43,00, Blend especial: R$ 37,50, Catuaí amarelo: R$ 55,00, Geisha: R$ 105,00, e Yirgacheffe: R$ 110,00.


In [ ]:
pergunta = chat_engine.chat('Voce pode me dar mais detalhes sobre o Catuai amarelo ?').response
print(pergunta)

O Catuai amarelo tem doçura máxima (5/5) e corpo médio-alto (4/5) com amargor baixo (1/5). Além disso, apresenta uma acidez expressiva e notas incríveis de pitanga, desviando do tradicional.


In [ ]:
pergunta = chat_engine.chat('Qual o preço dele ?').response
print(pergunta)

O preço do Catuai amarelo é R$ 55,00.


In [ ]:
chat_engine.chat_history

[ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='Quais graos estao disponiveis ?')]),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='Os grãos disponíveis são Bourbon vermelho, Bourbon amarelo, Blend especial (uma mistura de Bourbon amarelo e vermelho), Catuaí amarelo, Geisha e Yirgacheffe.')]),
 ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='Quais sao os preços dos graos ?')]),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='Os preços dos grãos são: Bourbon vermelho: R$ 41,00, Bourbon amarelo: R$ 43,00, Blend especial: R$ 37,50, Catuaí amarelo: R$ 55,00, Geisha: R$ 105,00, e Yirgacheffe: R$ 110,00.')]),
 ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='Voce p

In [ ]:
from llama_index.core.memory import ChatSummaryMemoryBuffer

In [ ]:
memory = ChatSummaryMemoryBuffer(llm=llms,token_limit=256)

In [ ]:
chat_engine= index.as_chat_engine(
    chat_mode = 'context',
    llm=llms,
    memory=memory,
    system_prompt=('''Voce é especialista em cafes da loja Serenato, uma loja online que vende graos de cafe
   torrados, sua funçao é tirar duvidas de forma simpatica e natural sobre os graos disponiveis''')
)

In [ ]:
response = chat_engine.chat('Ola')
print(response)

Olá! Bem-vindo à loja online Serenatto! Eu sou o especialista em cafés aqui. Estou aqui para ajudar a tirar suas dúvidas sobre nossos deliciosos grãos de café torrados. Qual é sua pergunta sobre nossos cafés?


In [ ]:
response = chat_engine.chat('Voce pode me dar mais detalhes sobre o catui amarelo')
print(response)

O Catuaí amarelo! É um dos nossos cafés mais exóticos e especiais. Ele possui uma doçura máxima (5/5) e um corpo médio-alto (4/5), com um amargor baixo (1/5). O que o torna realmente único é sua expressiva acidez e as notas maravilhosas de pitanga. É um café que foge do tradicional e é perfeito para quem busca uma experiência sensorial diferenciada.

O processo de produção do Catuaí amarelo é fermentado, cultivado em altitude de 1.200m, com torra média. Isso lhe confere um perfil sensorial muito interessante e agradável.

Se você é um amante de cafés que oferecem algo novo e emocionante, o Catuaí amarelo é uma ótima escolha. Você vai adorar sua complexidade e sua capacidade de surpreender!


In [ ]:
response = chat_engine.chat('qual o preço dele')
print(response)

O preço do Catuaí amarelo é de R$ 55,00 por pacote de 250g. É um investimento vale a pena para quem busca uma experiência de café única e de alta qualidade. Além disso, como todos os nossos cafés, o Catuaí amarelo tem uma nota SCA acima de 80, garantindo que você esteja recebendo um produto de alta qualidade.


In [ ]:
memory.get()

[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='It looks like we have a conversation started in Portuguese!\n\nTo summarize, the user greeted with "Ola" and the assistant responded with a warm welcome to the online store Serenatto, introducing themselves as a coffee expert. The assistant is ready to help answer any questions the user may have about their delicious roasted coffee beans.\n\nThe conversation so far:\n\nUser: Ola\nAssistant: Olá! Bem-vindo à loja online Serenatto! Eu sou o especialista em cafés aqui. Estou aqui para ajudar a tirar suas dúvidas sobre nossos deliciosos grãos de café torrados. Qual é sua pergunta sobre nossos cafés?\n\nThe user then asked for more details about the "Catui amarelo" coffee. The assistant responded by describing the coffee\'s characteristics, such as its high sweetness, medium-high body, and low bitterness. The assistant highlighted the coffee\'s unique acidity and notes of pitan

In [ ]:
#resetando o chat
chat_engine.reset()

In [ ]:
response = chat_engine.chat('qual o preço dele')
print(response)

Olá!

Você está perguntando sobre o preço de um dos nossos cafés especiais, certo?

Nós temos várias opções disponíveis, e os preços variam de acordo com a variedade. Aqui vão os preços de cada uma delas:

* Bourbon Vermelho: R$ 41,00
* Bourbon Amarelo: R$ 43,00
* Blend Especial (mistura de Bourbon Amarelo e Vermelho): R$ 37,50
* Catuaí Amarelo: R$ 55,00
* Geisha: R$ 105,00
* Yirgacheffe: R$ 110,00

Qual variedade você está interessado em saber mais sobre?


In [ ]:
# implementando a interface com o Gradio
!pip install gradio

In [ ]:
import gradio as gr

In [ ]:
# Criando a função para conversar com o chatbot
def converse_com_bot(message, chat_history):
    response = chat_engine.chat(message)

    if chat_history is None:
        chat_history = []

    chat_history.append({"role": "user", "content": message})
    chat_history.append({"role": "assistant", "content": response.response})

    return "", chat_history

# Criando a função para resetar o chatbot
def resetar_chat():
    chat_engine.reset()
    return []




In [ ]:
with gr.Blocks() as app:

    gr.Markdown('# Chatbot da Serenatto')
    chatbot = gr.Chatbot(type='messages')
    msg = gr.Textbox(label='Digite a sua mensagem')
    limpar = gr.Button('Limpar')

    msg.submit(converse_com_bot,[msg,chatbot],[msg,chatbot])
    limpar.click(resetar_chat,None,chatbot,queue=False)

    app.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a001597c67037ec64f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
